# Workshop Notebook 1: Build and Deploy a Model

For this workshop, let's pretend that you work for a real estate company wants to estimate the likely value of their listings portfolio. You have developed a model to predict the sale price of properties that the company has listed, based on data collected in the company's listings database.

In this set of exercises, you will build a model to predict house sale prices, and deploy it to Wallaroo.

Before we start, let's load some libraries that we will need for this notebook (note that this may not be a complete list).

In [ ]:
# preload needed libraries 

import wallaroo
from wallaroo.object import EntityNotFoundError

from IPython.display import display

# used to display DataFrame information without truncating
from IPython.display import display
import pandas as pd
pd.set_option('display.max_colwidth', None)

import json

import datetime
import time

# used for unique connection names

import string
import random


<hr/>

#### Exercise: Build a model

Use the house price data [here](https://github.com/WallarooLabs/Wallaroo_Tutorials/tree/main/notebooks_in_prod/data) to build a model to predict the sales price of homes based on the features in the data set. 

At the end of the exercise, you should have a notebook and possibly other artifacts to produce a model for predicting house prices. For the purposes of the exercise, please use a framework that can be converted to ONNX, such as scikit-learn or XGBoost.

**Note**

If you prefer to shortcut this step, you can use one of the pretrained model pickle files [here](https://drive.google.com/drive/folders/1Sg3wid53L8QDw3E9XavS55epJvFoQgiZ). **_NOTE: this link is currently internal_**.


In [ ]:
## Blank space for training model, if needed




## Getting Ready to deploy

Wallaroo natively supports models in the ONNX and Tensorflow frameworks, and other frameworks via containerization. For this exercise, we assume that you have a model that can be converted to the ONNX framework. The first steps to deploying in Wallaroo, then, is to convert your model to ONNX, and to add some extra functions to your processing modules so Wallaroo can call them. **_NOTE: we will want to replace this step with autoconversion when that is available_**.

<hr/>

#### Exercise: Convert your Model to ONNX

Take the model that you created in the previous exercises, and convert it to ONNX. If you need help, see the [Wallaroo Conversion Tutorials](https://docs.wallaroo.ai/wallaroo-tutorials/wallaroo-tutorials-conversion-tutorials/), or other [conversion documentation](https://github.com/onnx/tutorials#converting-to-onnx-format). 

At the end of this exercise, you should have your model as a standalone artifact, for example, a file called `model.onnx`.

**Note**

If you prefer to shortcut this exercise, you can use one of the pre-converted onnx files [here](https://drive.google.com/drive/folders/1Sg3wid53L8QDw3E9XavS55epJvFoQgiZ). **_NOTE: this link is currently internal_**.

In [ ]:
# Blank space to load for converting model, if needed




## Get ready to work with Wallaroo

Now that you have a model ready to go, you can log into Wallaroo and set up a **workspace** to organize your deployment artifacts. A Wallaroo workspace is place to organize the deployment artifacts for a project, and to collaborate with other team members. For more information, see [Wallaroo 101](https://docs.wallaroo.ai/wallaroo-101/). 


Logging into Wallaroo via the cluster's integrated JupyterLab is quite straightfoward:

```
# Login through local Wallaroo instance 
wl = wallaroo.Client()
```
See [the documentation](https://docs.wallaroo.ai/wallaroo-101/#open-a-connection-to-wallaroo) if you are logging into Wallaroo some other way.

Once you are logged in, you can create a workspace and set it as your working environment. To make the first exercise easier, here is a convenience function to get or create a workspace:

In [ ]:
# return the workspace called <name>, or create it if it does not exist.
# this function assumes your connection to wallaroo is called wl
def get_workspace(name):
    workspace = None
    for ws in wl.list_workspaces():
        if ws.name() == name:
            workspace= ws
    if(workspace == None):
        workspace = wl.create_workspace(name)
    return workspace

Then creating a workspace looks something like this:

```
# workspace names need to be globally unique, so add a random suffix to insure this
# especially important if the "main" workspace name is potentially a common one

suffix= ''.join(random.choice(string.ascii_lowercase) for i in range(4))
workspace_name = "my-workspace"+suffix

workspace = get_workspace(workspace_name)

# set your current workspace to the workspace that you just created
wl.set_current_workspace(workspace)

# optionally, examine your current workspace
wl.get_current_workspace()

```

<hr/>

#### Exercise: Log in and create a workspace

Log into wallaroo, and create a workspace for this workshop. Make sure you remember the name that you gave the workspace, as you will need it for later notebooks. Set that workspace to be your working environment.

At the end of the exercise, you should be in a new workspace to do further work.


In [ ]:
## Blank spot to log in and create workspace to work in



## Deploy a Simple Single-Step Pipeline

Once your model is in the ONNX format, and you have a workspace to work in, you can easily upload your model to Wallaroo's production platform with just a few lines of code. For example, if you have a model called `model.onnx`, and you wish to upload it to Wallaroo with the name `mymodel`, then upload the model as follows (once you are in the appropriate workspace):

```
my_model = wl.upload_model("mymodel", "model.onnx").configure()
```

The function `upload_model()` returns a handle to the uploaded model that you will continue to work with in the SDK.

Once the model has been uploaded, you can create a **pipeline** that contains the model. The pipeline is the mechanism that manages deployments. A pipeline contains a series of **steps** - sequential sets of models which take in the data from the preceding step, process it through the model, then return a result. Some pipelines can have just one step, while others may have multiple models with multiple steps or arranged for A/B testing. Deployed pipelines allocate resources and can then process data either through local files or through a **deployment URL**.

So for your model to accept inferences, you must add it to a pipeline. You can create a single step pipeline called `mypipeline` as follows:

```
# create the pipeline
my_pipeline = wl.build_pipeline("mypipeline").add_model_step(my_model)

# deploy the pipeline
my_pipeline = my_pipeline.deploy()
```

Deploying the pipeline means that it is ready to accept inferences. You can "turn off" the pipeline with the call `pipeline.undeploy()`.

**More Hints**

* `workspace = wl.get_current_workspace()` gives you a handle to the current workspace
* then `workspace.models()` will return a list of the models in the workspace
* and `workspace.pipelines()` will return a list of the pipelines in the workspace

<hr/>

#### Exercise: Upload and deploy your model

Upload and deploy the ONNX model that you created in the previous exercise. For simplicity, do any need pre-processing in the notebook.

At the end of the exercise, you should have a model and a deployed pipeline in your workspace.

In [ ]:
## blank space to upload model, and create the pipeline




## Sending Data to your Pipeline

In order to send data to your deployed pipeline, note that ONNX models generally expect their input as an array in a dictionary, keyed by input name. In Wallaroo, the default input name is "tensor". So (outside of Wallaroo), an ONNX model that expected three numeric values as its input would expect input data similar to the below:

```
# one datum
singleton = {'tensor': [[1, 2, 3]] }

# two datums
two_inputs = {'tensor': [[1, 2, 3], [4, 5, 6]] }
```

In the Wallaroo SDK, you can send a pandas data frame representation of this dictionary to the pipeline, via the `pipeline.infer()` method.

```
import pandas as pd

# one datum
sdf = pd.DataFrame(singleton)
sdf
#       tensor
# 0  [1, 2, 3]

# send the datum to a pipeline for inference
result = my_pipeline.infer(sdf)

# two datums
# Note that the value of 'tensor' must be a list, not a numpy array 
twodf = pd.DataFrame(two_inputs)
twodf
#      tensor
# 0  [1, 2, 3]
# 1  [4, 5, 6]

# send the data to a pipeline for inference
result = my_pipeline.infer(twodf)
```

To send data to a pipeline via the inference URL (for example, via CURL), you need the JSON representation of these data frames.

```
sdf.to_json(orient='records')
# '[{"tensor":[1,2,3]}]'

twodf.to_json(orient='records')
# '[{"tensor":[1,2,3]},{"tensor":[4,5,6]}]'
```

If the JSON data is in a file, you can send it to the pipeline from within the SDK via the `pipeline.infer_from_file()` method. 

In either case, a successful inference will return a data frame of inference results. The model inference(s) will be in the `column out.<outputname>`.


To help with the following exercises, here are some convenience functions you might find useful. These functions convert input data in standard tabular format (in a pandas data frame) to the pandas record format that the model expects.

In [ ]:
# pull a single datum from a data frame 
# and convert it to the format the model expects
def get_singleton(df, i):
    singleton = df.iloc[i,:].to_numpy().tolist()
    sdict = {'tensor': [singleton]}
    return pd.DataFrame.from_dict(sdict)


# pull a batch of data from a data frame
# and convert to the format the model expects
def get_batch(df, first=0, nrows=1):
    last = first + nrows
    batch = df.iloc[first:last, :].to_numpy().tolist()
    return pd.DataFrame.from_dict({'tensor': batch})

<hr/>

#### Exercise: Send data to your pipeline for inference.

Create some test data from the housing data and send it to the pipeline that you deployed in the previous exercise.  If you used the pre-provided models, then you can use `test_data.csv`, [here](https://drive.google.com/drive/folders/1Sg3wid53L8QDw3E9XavS55epJvFoQgiZ). **_NOTE: this link is currently internal_**.

* Start easy, with just one datum; retrieve the inference results. You can try small batches, as well.
* Examine the inference results; observe what the model prediction column is called; it should be of the form `out.<outputname>`.

For more hints about the different ways of sending data to the pipeline, and to see an example of the inference result format, see the ["Running Inferences" section of Wallaroo 101](https://docs.wallaroo.ai/wallaroo-101/#running-interfences).

At the end of the exercise, you should have a set of inference results that you got through the Wallaroo pipeline. 

**Don't forget to undeploy your pipeline after you are done**, to free up resources.



In [ ]:
##  blank space to create test data, and send some data to your model





## Congratulations!

You have now 

* successfully trained a model
* converted your model and uploaded it to Wallaroo
* created and deployed a simple single-step pipeline
* successfully send data to your pipeline for inference

In the next notebook, you will look at two different ways to evaluate your model against the real world environment.

